In [ ]:
from google.colab import files
uploaded = files.upload()  # Upload dataset.zip here


In [ ]:
import zipfile
import os

# Set path to where you want to unzip
unzip_dir = "/content/unzipped"
os.makedirs(unzip_dir, exist_ok=True)

# If you uploaded manually:
zip_path = list(uploaded.keys())[0]  # Automatically picks uploaded file

# Unzip the file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(unzip_dir)

print(f"Unzipped to: {unzip_dir}")


In [ ]:
pip install transformers datasets torch


In [ ]:
!pip install torch transformers datasets fsspec==2025.3.2 gcsfs


In [ ]:
!pip install torch transformers datasets fsspec==2025.3.2 gcsfs

from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline

# Public clinical BioBERT (you can fine-tune for NER if needed)
model_name = "emilyalsentzer/Bio_ClinicalBERT"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")  # fallback NER head

ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")

text = """
The patient was given 100 mg of aspirin to prevent blood clots. She has a history of hypertension and osteoarthritis.
"""

results = ner_pipeline(text)

print("🔍 Named Entities Detected:")
for ent in results:
    print(f"• {ent['word']} ({ent['entity_group']}) - Confidence: {ent['score']:.4f}")


In [ ]:
# ✅ Install dependencies
!pip install torch torchvision matplotlib

import os
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

# ✅ Data transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

# ✅ Load dataset (replace path with your local/kaggle dataset mount path)
# Example folder structure:
# ./brain_tumor_dataset/
#     ├── glioma_tumor/
#     ├── meningioma_tumor/
#     ├── pituitary_tumor/
#     └── no_tumor/
data_dir = '/content/unzipped'  # Replace with your actual path

dataset = datasets.ImageFolder(data_dir, transform=transform)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

# ✅ Use pre-trained ResNet for classification
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 4)  # 4 classes
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# ✅ Training setup
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# ✅ Training loop (minimal)
for epoch in range(5):  # Train for 5 epochs (increase for real results)
    model.train()
    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")


In [ ]:
!pip install torch torchvision matplotlib

import os
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt


In [ ]:
# ✅ Step 1: Install dependencies
!pip install torch torchvision matplotlib

# ✅ Step 2: Import required libraries
import os
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt

# ✅ Step 3: Define transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

# ✅ Step 4: Load dataset
data_dir = '/content/unzipped'  # Ensure your data is here
dataset = datasets.ImageFolder(data_dir, transform=transform)

# ✅ Step 5: Train/test split (80/20)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# ✅ Step 6: Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# ✅ Step 7: Load pretrained ResNet and modify classifier
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 4)  # 4 tumor classes
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# ✅ Step 8: Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# ✅ Step 9: Training loop
for epoch in range(5):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader):.4f}")

# ✅ Step 10: Evaluate on test set
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"\nTest Accuracy: {100 * correct / total:.2f}%")


In [ ]:
torch.save(model.state_dict(), 'resnet18_tumor_classifier.pth')


In [ ]:
model = models.resnet18(pretrained=False)
model.fc = nn.Linear(model.fc.in_features, 4)
model.load_state_dict(torch.load('resnet18_tumor_classifier.pth'))
model = model.to(device)
model.eval()


In [ ]:
import numpy as np

class_names = dataset.classes

def imshow(img):
    img = img / 2 + 0.5  # unnormalize if needed
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

# Display 4 test images with predictions
dataiter = iter(test_loader)
images, labels = next(dataiter)
images, labels = images.to(device), labels.to(device)
outputs = model(images)
_, preds = torch.max(outputs, 1)

for i in range(4):
    imshow(images[i].cpu())
    print(f'Actual: {class_names[labels[i]]}, Predicted: {class_names[preds[i]]}')


In [ ]:
losses = []

for epoch in range(5):
    ...
    losses.append(running_loss / len(train_loader))

# Plot loss curve
plt.plot(range(1, 6), losses)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.show()


In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

y_true = []
y_pred = []

model.eval()
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())

cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
disp.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.show()


In [ ]:
# Unfreeze last few layers for fine-tuning
for name, param in model.named_parameters():
    if "layer4" in name or "fc" in name:  # layer4 is the last ResNet block
        param.requires_grad = True
    else:
        param.requires_grad = False


In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.1, contrast=0.1),
    transforms.ToTensor()
])


In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_true, y_pred, target_names=class_names))


In [ ]:
!pip install gradio


In [ ]:
!pip install gradio -q
import gradio as gr
from PIL import Image


In [ ]:
import torchvision.transforms as transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])


In [ ]:
import torch
import torch.nn as nn
from torchvision.models import resnet18

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = resnet18(weights=None)
model.fc = nn.Linear(model.fc.in_features, 4)  # 4 classes
model.load_state_dict(torch.load('resnet18_tumor_classifier.pth', map_location=device))
model = model.to(device)
model.eval()


In [ ]:
class_names = ['glioma', 'meningioma', 'pituitary', 'no_tumor']


In [ ]:
def predict_tumor(image):
    image = transform(image.convert("RGB")).unsqueeze(0).to(device)
    with torch.no_grad():
        output = model(image)
        _, predicted = torch.max(output, 1)
    return class_names[predicted.item()]


In [ ]:
gr.Interface(
    fn=predict_tumor,
    inputs=gr.Image(type="pil"),
    outputs="label",
    title="Tumor Type Classifier",
    description="Upload a brain scan image to predict the tumor type"
).launch(share=True)  # Use share=True for Colab
